In [329]:
import pathlib
import numpy as np
import pandas as pd
import json
import re
h = pathlib.Path.home()
pet_data = open(f"{h}/downloads/PET 2.txt",'r').readlines()

In [330]:
def petdata(pet_string):
    d = json.loads(pet_string)
    data_col = d['name'].replace(',','').replace('(','').replace(')','')
    values = np.array(d['data'])
    dates = values[:,0].astype(int)
    vals = values[:,1].astype(float)
    return data_col,dates,vals


In [331]:
def padd_arr(pet_string,vals_len):
    arr = petdata(pet_string)
    col_name = arr[0]
    vals = arr[2]
    if len(vals)>=vals_len:
        return col_name,vals[:vals_len]
    new_vals = np.pad(vals,(0,vals_len-len(vals)),'constant',constant_values=(0,np.nan))
    return col_name,new_vals

In [332]:
padd_arr(pet_data[700],1540)

('Midwest PADD 2 Imports of Crude Oil and Petroleum Products 4 Week Avg',
 array([2702., 2628., 2459., ...,   nan,   nan,   nan]))

In [333]:
from tqdm import tqdm_notebook
data_col,dates,vals = petdata(pet_data[0])
vals_len = len(vals)
df_pet = pd.DataFrame({'date':dates,data_col:vals})

for i in tqdm_notebook(range(1,len(pet_data))):
    try:
        data_col,vals = padd_arr(pet_data[i],vals_len)
    except:
        continue
    if 'weekly' not in data_col.lower():
        continue
    if '4 week avg' in data_col.lower():
        continue
    
    df_pet[data_col] = vals

df_pet


/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



,date,Florida Midgrade Conventional Retail Gasoline Prices Weekly,Boston MA Premium All Formulations Retail Gasoline Prices Weekly,San Francisco CA Premium Reformulated Retail Gasoline Prices Weekly,New York Harbor Premium Reformulated Retail Gasoline Prices Weekly,Houston TX Premium Reformulated Retail Gasoline Prices Weekly,West Coast PADD 5 Except California Premium Conventional Retail Gasoline Prices Weekly,West Coast PADD 5 Except California Midgrade Reformulated Retail Gasoline Prices Weekly,Los Angeles CA Midgrade Reformulated Retail Gasoline Prices Weekly,San Francisco CA Midgrade Reformulated Retail Gasoline Prices Weekly,...,Rocky Mountain PADD 4 Ending Stocks of Reformulated GTAB Gasoline Blending Components Weekly,West Coast PADD 5 Ending Stocks of Reformulated GTAB Gasoline Blending Components Weekly,U.S. Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,East Coast PADD 1 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,Midwest PADD 2 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,Gulf Coast PADD 3 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,Rocky Mountain PADD 4 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,West Coast PADD 5 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,U.S. Ending Stocks of Reformulated RBOB with Ether Gasoline Blending Components Weekly,East Coast PADD 1 Ending Stocks of Reformulated RBOB with Ether Gasoline Blending Components Weekly
0,20200817,2.395,2.631,3.578,2.632,2.425,3.028,2.639,3.286,3.415,...,0.0,0.0,48528.0,19625.0,5544.0,11841.0,0.0,11518.0,0.0,0.0
1,20200810,2.438,2.625,3.557,2.633,2.425,3.023,2.638,3.269,3.397,...,0.0,0.0,48324.0,19279.0,5128.0,11693.0,0.0,12224.0,0.0,0.0
2,20200803,2.469,2.630,3.581,2.646,2.438,3.034,2.651,3.284,3.414,...,0.0,0.0,47814.0,18463.0,5457.0,10957.0,0.0,12937.0,0.0,0.0
3,20200727,2.372,2.632,3.555,2.653,2.451,3.022,2.655,3.279,3.370,...,0.0,0.0,49471.0,19028.0,5494.0,11788.0,0.0,13161.0,0.0,0.0
4,20200720,2.386,2.636,3.508,2.667,2.458,3.019,2.662,3.260,3.313,...,0.0,0.0,49241.0,18567.0,5112.0,11822.0,0.0,13740.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,20030623,1.569,1.692,2.100,1.696,1.561,1.891,1.749,1.868,2.004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
896,20030616,1.579,1.698,2.069,1.702,1.579,1.887,1.748,1.866,1.972,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
897,20030609,1.550,1.683,1.983,1.710,1.571,1.833,1.666,1.799,1.878,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
898,20030602,1.540,1.698,2.012,1.726,1.554,1.834,1.679,1.839,1.914,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [334]:
df_pet.to_csv('./temp_folder/df_pet.csv',index=None)

In [335]:
lower_cols = [c.lower() for c in df_pet.columns.values]
df_pet_lower = df_pet.copy()
df_pet_lower.columns = lower_cols
df_pet_lower.to_csv('./temp_folder/df_pet_lower.csv',index=None)

In [336]:
cols = df_pet_lower.columns.values
df_reports = pd.DataFrame({'report':cols[np.where(cols!='date')]})
df_reports['colnum'] = df_reports.index.values
df_reports.to_csv('./temp_folder/df_pet_columns.csv',index=False)

In [337]:
df_reports[~df_reports.report.str.contains('stocks')]

,report,colnum
0,florida midgrade conventional retail gasoline prices weekly,0
1,boston ma premium all formulations retail gasoline prices weekly,1
2,san francisco ca premium reformulated retail gasoline prices weekly,2
3,new york harbor premium reformulated retail gasoline prices weekly,3
4,houston tx premium reformulated retail gasoline prices weekly,4
...,...,...
1109,east coast padd 1 imports of reformulated rbob with ether gasoline blending components weekly,1109
1110,midwest padd 2 rbob with ether gasoline blending components imports weekly,1110
1111,gulf coast padd 3 imports of reformulated rbob with ether gasoline blending components weekly,1111
1112,rocky mountain padd 4 rbob with ether gasoline blending components imports weekly,1112


In [338]:
reports_text = "\n".join(df_reports.report.values)


In [339]:
print(reports_text)

florida midgrade conventional retail gasoline prices weekly
boston ma premium all formulations retail gasoline prices weekly
san francisco ca premium reformulated retail gasoline prices weekly
new york harbor premium reformulated retail gasoline prices weekly
houston tx premium reformulated retail gasoline prices weekly
west coast padd 5 except california premium conventional retail gasoline prices weekly
west coast padd 5 except california midgrade reformulated retail gasoline prices weekly
los angeles ca midgrade reformulated retail gasoline prices weekly
san francisco ca midgrade reformulated retail gasoline prices weekly
new york harbor midgrade reformulated retail gasoline prices weekly
houston tx midgrade reformulated retail gasoline prices weekly
west coast padd 5 except california midgrade conventional retail gasoline prices weekly
seattle wa midgrade conventional retail gasoline prices weekly
seattle wa all grades conventional retail gasoline prices weekly
seattle wa regular c

In [340]:
import spacy
import pytextrank

# example text
text = "Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types."
text = reports_text
# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_sm")

# add PyTextRank to the spaCy pipeline
tr = pytextrank.TextRank()
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

doc = nlp(text)

# examine the top-ranked phrases in the document
for p in doc._.phrases:
    print("{:.4f} {:5d}  {}".format(p.rank, p.count, p.text))
#     print(p.chunks)

0.1469    35  conventional retail gasoline prices
0.1446    87  retail gasoline prices
0.1440     2  reformulated retail gasoline prices
0.1415     5  regular conventional retail gasoline prices
0.1398     1  u.s. regular conventional retail gasoline prices
0.1388     4  regular reformulated retail gasoline prices
0.1381     1  u.s. premium conventional retail gasoline prices
0.1377     1  u.s. regular reformulated retail gasoline prices
0.1372     1  west coast regular conventional retail gasoline prices
0.1371     1  gulf coast regular conventional retail gasoline prices
0.1370     1  east coast regular conventional retail gasoline prices
0.1363     1  west coast midgrade conventional retail gasoline prices
0.1361     1  gulf coast midgrade conventional retail gasoline prices
0.1355     1  west coast premium conventional retail gasoline prices
0.1353     1  gulf coast premium conventional retail gasoline prices
0.1352     1  east coast premium conventional retail gasoline prices
0.13

In [341]:
ranks = []
counts = []
texts = []
for p in doc._.phrases:
    ranks.append(p.rank)
    counts.append(p.count)
    texts.append(p.text)



In [342]:
df_phrases_byrank = pd.DataFrame({'p_rank':ranks,'p_count':counts,'p_text':texts})
df_phrases_bycount = df_phrases_byrank.sort_values('p_count',ascending=False)


In [343]:
pd.set_option('display.max_colwidth', 0)
df_phrases_byrank

,p_rank,p_count,p_text
0,0.146925,35,conventional retail gasoline prices
1,0.144554,87,retail gasoline prices
2,0.144008,2,reformulated retail gasoline prices
3,0.141506,5,regular conventional retail gasoline prices
4,0.139849,1,u.s. regular conventional retail gasoline prices
...,...,...,...
477,0.008182,5,4 oxygenate
478,0.008130,1,3 gross
479,0.007795,7,2 rbob
480,0.007006,6,all countries


In [344]:
import dash_table
import dash
import dash_html_components as html

def run_reports(df,displayed_rows=100):

#     displayed_rows =10
    dt_reports = dash_table.DataTable(
        data=df.to_dict('records'),
        columns = [{'name':c,'id':c} for c in df.columns.values],
        page_size=displayed_rows,
        page_action='native',
        style_table={
            'overflowY':'auto',
#             'height': f'{35*(displayed_rows+1)+1}px'
        },
        style_cell={
            'whiteSpace': 'normal',
            'height': 'auto',
            'minWidth': '50px'
        },
        filter_action='native',
#         fixed_rows={'headers': True},
    )
    app = dash.Dash()
    s = {'display':'grid'}
    app.layout=html.Div(
        [dt_reports],
        style=s
    )
    app.run_server()
        

In [74]:
run_reports(df_phrases_bycount[['p_count','p_text']])

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 i

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Aug/2020 11:01:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2020 11:01:12] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2020 11:01:12] "GET /_dash-layout HTTP/1.1" 200 -


In [345]:
df_phrases_byrank.to_csv(f'{h}/downloads/df_phrases_byrank.csv',index=False)

In [346]:

s = "padd.{1,2}[1-5][a-e]{0,1}"
df2 = df_phrases_byrank[df_phrases_byrank.p_text.str.contains(s)]
finds = [re.findall(s,v)[0] for v  in df2.p_text.values]
splits = [re.split(s,v) for v  in df2.p_text.values]
df2['token'] = finds
df2['left'] = [s[0] for s in splits]
df2['right'] = [None if len(s)<2 else s[1] for s in splits]

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [347]:
df2

,p_rank,p_count,p_text,token,left,right
58,0.122298,1,new england padd 1a premium conventional retail gasoline prices,padd 1a,new england,premium conventional retail gasoline prices
109,0.097362,1,central atlantic padd 1b premium conventional retail gasoline prices,padd 1b,central atlantic,premium conventional retail gasoline prices
128,0.088393,1,weekly rocky mountain padd 4 propane residential price weekly,padd 4,weekly rocky mountain,propane residential price weekly
130,0.087076,1,east coast padd 1 propane residential price weekly,padd 1,east coast,propane residential price weekly
140,0.081777,1,new england padd 1a propane residential price weekly,padd 1a,new england,propane residential price weekly
172,0.074039,1,west coast padd 5 commercial crude oil imports,padd 5,west coast,commercial crude oil imports
175,0.073314,1,east coast padd 1 commercial crude oil imports,padd 1,east coast,commercial crude oil imports
179,0.072295,1,gulf coast padd 3 commercial crude oil imports,padd 3,gulf coast,commercial crude oil imports
210,0.069362,1,east coast padd 1 propane wholesale/resale price weekly,padd 1,east coast,propane wholesale/resale price weekly
214,0.068660,1,east coast padd 1 refiner net production,padd 1,east coast,refiner net production


In [348]:
cols = ['california regular reformulated retail gasoline prices weekly','california regular reformulated retail gasoline prices']
df_pet2 = df_pet.copy()
df_pet2.columns = [v.lower()for v in df_pet2.columns.values]

In [349]:
df_pet2[cols]

KeyError: "['california regular reformulated retail gasoline prices'] not in index"

### get padd info

In [350]:
cols = [c for c in df_pet.columns.values if len(re.findall('padd [0-9][a-z]{0,1}',c.lower()))>0]
cols = [c for c in cols if c!='date']
# cols = [c for c in cols if 'all' not in c.lower()]

In [351]:
cols

['West Coast PADD 5 Except California Premium Conventional Retail Gasoline Prices Weekly',
 'West Coast PADD 5 Except California Midgrade Reformulated Retail Gasoline Prices Weekly',
 'West Coast PADD 5 Except California Midgrade Conventional Retail Gasoline Prices Weekly',
 'West Coast PADD 5 Except California No 2 Diesel Retail Prices Weekly',
 'West Coast PADD 5 Except California All Grades Reformulated Retail Gasoline Prices Weekly',
 'West Coast PADD 5 Except California All Grades Conventional Retail Gasoline Prices Weekly',
 'West Coast PADD 5 Except California Regular Reformulated Retail Gasoline Prices Weekly',
 'West Coast PADD 5 Except California Regular Conventional Retail Gasoline Prices Weekly',
 'West Coast PADD 5 Except California Premium Reformulated Retail Gasoline Prices Weekly',
 'Lower Atlantic PADD 1C No 2 Diesel Ultra Low Sulfur 0-15 ppm Retail Prices Weekly',
 'West Coast PADD 5 Except California No 2 Diesel Ultra Low Sulfur 0-15 ppm Retail Prices Weekly',
 'Cent

In [352]:
df_pet[['date']+cols]

,date,West Coast PADD 5 Except California Premium Conventional Retail Gasoline Prices Weekly,West Coast PADD 5 Except California Midgrade Reformulated Retail Gasoline Prices Weekly,West Coast PADD 5 Except California Midgrade Conventional Retail Gasoline Prices Weekly,West Coast PADD 5 Except California No 2 Diesel Retail Prices Weekly,West Coast PADD 5 Except California All Grades Reformulated Retail Gasoline Prices Weekly,West Coast PADD 5 Except California All Grades Conventional Retail Gasoline Prices Weekly,West Coast PADD 5 Except California Regular Reformulated Retail Gasoline Prices Weekly,West Coast PADD 5 Except California Regular Conventional Retail Gasoline Prices Weekly,West Coast PADD 5 Except California Premium Reformulated Retail Gasoline Prices Weekly,...,Midwest PADD 2 Ending Stocks of Reformulated GTAB Gasoline Blending Components Weekly,Gulf Coast PADD 3 Ending Stocks of Reformulated GTAB Gasoline Blending Components Weekly,Rocky Mountain PADD 4 Ending Stocks of Reformulated GTAB Gasoline Blending Components Weekly,West Coast PADD 5 Ending Stocks of Reformulated GTAB Gasoline Blending Components Weekly,East Coast PADD 1 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,Midwest PADD 2 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,Gulf Coast PADD 3 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,Rocky Mountain PADD 4 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,West Coast PADD 5 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,East Coast PADD 1 Ending Stocks of Reformulated RBOB with Ether Gasoline Blending Components Weekly
0,20200817,3.028,2.639,2.884,2.586,2.413,2.680,2.351,2.608,2.840,...,0.0,0.0,0.0,0.0,19625.0,5544.0,11841.0,0.0,11518.0,0.0
1,20200810,3.023,2.638,2.877,2.588,2.415,2.674,2.353,2.602,2.840,...,0.0,0.0,0.0,0.0,19279.0,5128.0,11693.0,0.0,12224.0,0.0
2,20200803,3.034,2.651,2.881,2.592,2.426,2.682,2.364,2.610,2.856,...,0.0,0.0,0.0,0.0,18463.0,5457.0,10957.0,0.0,12937.0,0.0
3,20200727,3.022,2.655,2.873,2.586,2.429,2.677,2.366,2.606,2.858,...,0.0,0.0,0.0,0.0,19028.0,5494.0,11788.0,0.0,13161.0,0.0
4,20200720,3.019,2.662,2.865,2.597,2.434,2.677,2.372,2.607,2.865,...,0.0,0.0,0.0,0.0,18567.0,5112.0,11822.0,0.0,13740.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,20030623,1.891,1.749,1.767,NaN,1.688,1.705,1.650,1.665,1.850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
896,20030616,1.887,1.748,1.764,NaN,1.686,1.698,1.648,1.656,1.849,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
897,20030609,1.833,1.666,1.706,NaN,1.607,1.641,1.570,1.600,1.765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
898,20030602,1.834,1.679,1.700,NaN,1.617,1.636,1.579,1.593,1.785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [353]:
def transform_cols(cols):
    padd_regex = "(padd [0-9][a-z]{0,1}){1,2}"
    padd_regions = []
    padds = []
    padd_categories = []
    padd_product_types = []
    for c in cols:
    #     padd = re.findall(padd_regex,c.lower())[0]
    #     split1 = re.split('padd [0-9][a-z]{0,1}',c.lower())
        split1 = re.split(padd_regex,c.lower())
        padd = ''.join(split1[1:-1])
        padd_region = split1[0].strip()
        split2 = split1[-1].strip().lower().split('of')
        padd_category = split2[0].strip()
        padd_product_type = '' if len(split2)<2 else split2[1].strip()
        padds.append(padd)
        padd_regions.append(padd_region)
        padd_categories.append(padd_category)
        padd_product_types.append(padd_product_type)

    df_padds = pd.DataFrame(
        {
            'padd':padds,
            'padd_region':padd_regions,
            'padd_category':padd_categories,
            'padd_product_type':padd_product_types,
            'col':cols
        }
    )
    return df_padds

In [660]:
exp0 = '''
padd_regex = "(padd [0-9][a-z]{0,1}){1,2}"
split1 = re.split(padd_regex,c.lower())
padd = ''.join(split1[1:-1])
padd_region = split1[0].strip()
split2 = split1[-1].strip().lower().split('of')
padd_category = split2[0].strip()
padd_product_type = '' if len(split2)<2 else split2[1].strip()
print(padd)
print(padd_region)
print(padd_category)
print(padd_product_type)
'''

exp_padd = '''
"(padd [0-9][a-z]{0,1}){1,2}"
re.split(val,col.lower())
''.join(val[1:-1])
'''

exp_padd_region = '''
"(padd [0-9][a-z]{0,1}){1,2}"
re.split(val,col.lower())
val[0].strip()
'''

exp_padd_category = '''
"(padd [0-9][a-z]{0,1}){1,2}"
re.split(val,col.lower())
val[-1].strip().lower().split('of')[0]'''

exp_padd_source = '''
"(padd [0-9][a-z]{0,1}){1,2}"
re.split(val,col.lower())
val[-1].strip().lower().split('of')[-1]'''

exp3 = '''
r = "(padd [0-9][a-z]{0,1}){1,2}"
s = re.findall(r,col.lower())[0]
print(r,s)
'''

cols[0]

'Midwest PADD 2 Imports of Other Reformulated Motor Gasoline Weekly'

In [656]:
def do_eval(code,cols):
    arr = []
    for col in cols:
        val = None
        try:
            for expr in code.split('\n'):
                if len(expr)>0:
                    val = eval(expr)
        except:
            continue
        arr.append(val)
    return arr

def eval_df(dd,cols):
    ddd = {k:do_eval(dd[k],cols) for k in dd.keys()}
    df = pd.DataFrame(ddd)
    return df


In [657]:
dd = {'padd':exp_padd,'padd_region':exp_padd_region,'exp_padd_category':exp_padd_category}
eval_df(dd,cols)


# padds = do_eval(exp_padd)
# padd_regions  = do_eval(exp_padd_region)
# padd_categories  = do_eval(exp_padd_category)
# df_padds = pd.DataFrame() 

,padd,padd_region,exp_padd_category
0,,date,[date]
1,,florida midgrade conventional retail gasoline prices weekly,[florida midgrade conventional retail gasoline prices weekly]
2,,boston ma premium all formulations retail gasoline prices weekly,[boston ma premium all formulations retail gasoline prices weekly]
3,,san francisco ca premium reformulated retail gasoline prices weekly,[san francisco ca premium reformulated retail gasoline prices weekly]
4,,new york harbor premium reformulated retail gasoline prices weekly,[new york harbor premium reformulated retail gasoline prices weekly]
...,...,...,...
1128,padd 3,gulf coast,"[ending stocks , reformulated rbob with alcohol gasoline blending components weekly]"
1129,padd 4,rocky mountain,"[ending stocks , reformulated rbob with alcohol gasoline blending components weekly]"
1130,padd 5,west coast,"[ending stocks , reformulated rbob with alcohol gasoline blending components weekly]"
1131,,u.s. ending stocks of reformulated rbob with ether gasoline blending components weekly,"[u.s. ending stocks , reformulated rbob with ether gasoline blending components weekly]"


In [658]:
cols = [c for c in df_pet.columns.values if len(re.findall('padd [0-9][a-z]{0,1}',c.lower()))>0]
cols = [c for c in cols if c!='date']
cols = [c for c in cols if 'of' in c.lower()]


df_padds = transform_cols(cols)

In [659]:
df_padds.padd.unique()

array(['padd 2', 'padd 3', 'padd 4', 'padd 5', 'padd 1',
       'padd 4 and west coast padd 5', 'padd 1b', 'padd 1c', 'padd 2r',
       'padd 1a', 'padd 4 and padd 5'], dtype=object)

In [262]:
df_padds.padd_region.unique()

array(['rocky mountain', 'west coast', 'new england', 'central atlantic',
       'lower atlantic', 'east coast', 'midwest', 'gulf coast',
       'rocky mountains', ''], dtype=object)

In [257]:
df_padds.padd_category.unique()

array(['ending stocks', 'refiner and blender net production',
       'refiner net input', 'imports',
       'refiner blender and gas plant net production',
       'ending stocks excluding spr', 'refiner and blender net input',
       'efiner and blender net production', 'percent utilization',
       'refiner net production', 'blender net production',
       'oxygenate plant production', 'imports from  all countries',
       'imports from',
       'ending stocks excluding spr and including lease stock'],
      dtype=object)

In [704]:
PAD_REGEX = "(padd [0-9][a-z]{0,1}){1,2}"
PRODUCT_TYPE = "(gasoline)|(no 2[.]{0,1} diesel)|(propane( and propylene){0,1})|(crude)|(no. 2 heating oil)|(oxygenate)"
GRADE = "(regular)|(premium)|(midgrade)|(all grades)"
UNIT_TYPE = '(inputs)|(net input)|(capacity)|(net production)|(production)|(imports)|(ending stocks)|(utilization)'
PROCESSOR_TYPE = '(refiner and blender)|(refiner)|(blender)|(oxygenate plant)|(gas plant)'

def has_padd(col):
    return len(re.findall(PAD_REGEX,col.lower()))>0

def process_pad(cols,f,exclude_words=[]):
    vals = []
    for c in cols:
        if not has_padd(c):
            continue
        if any([ew in c for ew in exclude_words]):
            continue
        val = f(c)
        vals.append(val.strip().rstrip())
    return vals

def _padd(c):
    val = re.split(PAD_REGEX,c.lower())
    val = ''.join(val[1:-1])
    return val.strip().lstrip()

def get_padd(cols):
    return process_pad(cols,_padd)

def _padd_region(c):
    val = re.split(PAD_REGEX,c.lower())
    val = val[0].strip()
    g = GRADE
    val = re.split(g,val)[0]
    return val.strip().lstrip()

def get_padd_region(cols,**args):
    return process_pad(cols,_padd_region,**args)

def _pad_category(c):
    val = re.split(PAD_REGEX,c.lower())
    val = val[-1].strip().lower().split('of')[0]
    return val.strip().lstrip()

def get_padd_category(cols,**args):
    return process_pad(cols,_pad_category,**args)

def get_padd_category2(cols):
    vals = []
    for c in cols:
        if not has_padd(c):
            vals.append(None)
            continue
        val = re.split(PAD_REGEX,c.lower())
        val = val[-1].strip().lower().split('of')[0]
        if 'price' in val:
            vals.append(None)
            continue
        vals.append(val.strip().rstrip())
    return vals

def _pad_source(c):
    val = re.split(PAD_REGEX,c.lower())
    val = val[-1].strip().lower().split('of')[-1]
    return val.strip().lstrip()

def get_padd_source(cols,**args):
    return process_pad(cols,_pad_source,**args)


def get_padd_gasoline_type(cols,**args):
    vals = get_padd_source(cols,**args)
    vals = [re.split('gasoline',v)[0].strip().lstrip() for v in vals]
    return vals

def _padd_processor_type(c):
    val = re.split(PAD_REGEX,c.lower())
    val = ''.join(val[1:-1])
    return val.strip().lstrip()

def get_padd_processor_type(cols,**args):
    return process_pad(cols,_padd_processor_type,**args)

def get_ids(cols):
    def _findall(r,c):
        return ''.join([v for v in re.split(r,c.lower())[1:-1] if v is not None])
        l = re.findall(attribute_regexes[i],c)
        if l is None or len(l)<1:
            return ''
        if type(l[0]) == tuple:
            l = l[0]
        return ' '.join(l).strip().lstrip()
        
    all_attributes = {}
    attribute_regexes = [PAD_REGEX,PRODUCT_TYPE,GRADE,UNIT_TYPE,PROCESSOR_TYPE]
    attribute_names = ['PAD_REGEX','PRODUCT_TYPE','GRADE','UNIT_TYPE','PROCESSOR_TYPE']
    for i in range(len(attribute_regexes)):
        r = attribute_regexes[i]
        alist = [_findall(r,c) for c in cols]
        all_attributes[attribute_names[i]] = alist
    all_attributes['col'] = cols
    return pd.DataFrame(all_attributes)

def create_unique_values(df_col_ids):
    df = df_col_ids[[c for c in df_col_ids.columns.values if c!='col']]
    # create a list of tuples, where each tuple has a column name and a list of unique values
    vals = [(c,list(df[c].unique())) for c in df.dtypes[(df.dtypes=='object') | (df.dtypes=='str')].index]
    # if there are no tuples in the list, return an empty DataFrame
    if len(vals)<1:
        return pd.DataFrame()
    # create a one row DataFrame with the first column, and the unique values for that column
    df_unique = pd.DataFrame({vals[0][0]:vals[0][1]})

    # Loop through the remaining columns, using pd. concat to add new columns
    #   to df_unique, which will automatically broadcast each column so that 
    #   all columns have the same length
    for v in vals[1:]:
        df_unique = pd.concat([df_unique,pd.DataFrame({v[0]:v[1]})],axis=1)
    df_unique = df_unique.fillna('')
    df_unique = df_unique[df_unique.apply(lambda r:len(''.join(r))>0,axis=1)]
    df_unique.index = list(range(len(df_unique)))
    return df_unique


In [707]:
df_cols = get_ids(df_pet_lower.columns.values)
df_cols

,PAD_REGEX,PRODUCT_TYPE,GRADE,UNIT_TYPE,PROCESSOR_TYPE,col
0,,,,,,date
1,,gasoline,midgrade,,,florida midgrade conventional retail gasoline prices weekly
2,,gasoline,premium,,,boston ma premium all formulations retail gasoline prices weekly
3,,gasoline,premium,,,san francisco ca premium reformulated retail gasoline prices weekly
4,,gasoline,premium,,,new york harbor premium reformulated retail gasoline prices weekly
...,...,...,...,...,...,...
1128,padd 3,gasoline,,ending stocks,,gulf coast padd 3 ending stocks of reformulated rbob with alcohol gasoline blending components weekly
1129,padd 4,gasoline,,ending stocks,,rocky mountain padd 4 ending stocks of reformulated rbob with alcohol gasoline blending components weekly
1130,padd 5,gasoline,,ending stocks,,west coast padd 5 ending stocks of reformulated rbob with alcohol gasoline blending components weekly
1131,,gasoline,,ending stocks,,u.s. ending stocks of reformulated rbob with ether gasoline blending components weekly


In [708]:
dfc = create_unique_values(df_cols)
dfc

,PAD_REGEX,PRODUCT_TYPE,GRADE,UNIT_TYPE,PROCESSOR_TYPE
0,padd 5,gasoline,midgrade,imports,refiner blender and gas plant
1,padd 1c,no 2 diesel,premium,net production,refiner and blender
2,padd 1b,propane,all grades,net input,blender
3,padd 1a,propane and propylene and propylene,regular,ending stocks,refiner
4,padd 2,crude,,production,oxygenate plant
5,padd 3,oxygenate,,utilization of refinery operable capacity,
6,padd 4,no. 2 heating oil,,capacity,
7,padd 1,gasoline non-oxygenate,,inputs,
8,padd 4 and west coast padd 5,,,,
9,padd 4 and padd 5,,,,


In [717]:
df_pet_lower.iloc[0:10][['date']]

,date
0,20200817
1,20200810
2,20200803
3,20200727
4,20200720
5,20200713
6,20200706
7,20200629
8,20200622
9,20200615


In [731]:
pad_regex_list = ['padd 1','padd 4']
pad_regex_screen = df_cols.PAD_REGEX.isin(pad_regex_list)
pad_prod_type_list = ['gasoline','propane']
pad_prod_type_screen = df_cols.PRODUCT_TYPE.isin(pad_prod_type_list)
pad_unit_type_list = ['ending stocks']
pad_unit_type_screen = df_cols.UNIT_TYPE.isin(pad_unit_type_list)
cols_to_view = list(df_cols[pad_regex_screen & pad_prod_type_screen & pad_unit_type_screen].col.values)
cols_to_view = ['date'] + cols_to_view
df_screen = df_pet_lower[cols_to_view]
df_screen

,date,east coast padd 1 ending stocks of conventional motor gasoline greater than ed55 weekly,rocky mountain padd 4 ending stocks of conventional motor gasoline greater than ed55 weekly,east coast padd 1 ending stocks of conventional motor gasoline ed55 and lower weekly,rocky mountain padd 4 ending stocks of conventional motor gasoline ed55 and lower weekly,east coast padd 1 ending stocks of motor gasoline blending components rbob weekly,rocky mountain padd 4 ending stocks of motor gasoline blending components rbob weekly,east coast padd 1 ending stocks of finished motor gasoline weekly,rocky mountain padd 4 ending stocks of finished motor gasoline weekly,east coast padd 1 ending stocks of conventional gtab gasoline blending components weekly,...,east coast padd 1 ending stocks of reformulated motor gasoline weekly,rocky mountain padd 4 ending stocks of reformulated motor gasoline weekly,east coast padd 1 ending stocks of total gasoline weekly,rocky mountain padd 4 ending stocks of total gasoline weekly,rocky mountain padd 4 rbob with ether gasoline blending components ending stocks weekly,east coast padd 1 ending stocks of reformulated gtab gasoline blending components weekly,rocky mountain padd 4 ending stocks of reformulated gtab gasoline blending components weekly,east coast padd 1 ending stocks of reformulated rbob with alcohol gasoline blending components weekly,rocky mountain padd 4 ending stocks of reformulated rbob with alcohol gasoline blending components weekly,east coast padd 1 ending stocks of reformulated rbob with ether gasoline blending components weekly
0,20200817,0.0,0.0,61.0,81.0,19783.0,0.0,4615.0,1652.0,2336.0,...,20.0,0.0,68255.0,7081.0,0.0,0.0,0.0,19625.0,0.0,0.0
1,20200810,0.0,0.0,26.0,46.0,18462.0,0.0,4286.0,1799.0,1877.0,...,33.0,0.0,68498.0,7233.0,0.0,0.0,0.0,19279.0,0.0,0.0
2,20200803,0.0,0.0,26.0,52.0,20008.0,0.0,4264.0,1870.0,1754.0,...,29.0,0.0,67999.0,7535.0,0.0,0.0,0.0,18463.0,0.0,0.0
3,20200727,0.0,0.0,26.0,58.0,19859.0,0.0,4835.0,2044.0,1289.0,...,23.0,0.0,67688.0,7689.0,0.0,0.0,0.0,19028.0,0.0,0.0
4,20200720,0.0,0.0,26.0,52.0,22565.0,0.0,4398.0,1957.0,1429.0,...,30.0,0.0,71092.0,7625.0,0.0,0.0,0.0,18567.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,20030623,NaN,NaN,NaN,NaN,NaN,NaN,51749.0,4266.0,NaN,...,19051.0,0.0,59860.0,5565.0,NaN,NaN,NaN,NaN,NaN,NaN
896,20030616,NaN,NaN,NaN,NaN,NaN,NaN,50332.0,4370.0,NaN,...,19945.0,0.0,58733.0,5672.0,NaN,NaN,NaN,NaN,NaN,NaN
897,20030609,NaN,NaN,NaN,NaN,NaN,NaN,48579.0,4517.0,NaN,...,18462.0,0.0,56649.0,6005.0,NaN,NaN,NaN,NaN,NaN,NaN
898,20030602,NaN,NaN,NaN,NaN,NaN,NaN,48340.0,4472.0,NaN,...,16844.0,0.0,56099.0,5925.0,NaN,NaN,NaN,NaN,NaN,NaN


In [737]:
from dashapp import dashapp2 as dashapp
from plotly.offline import iplot

ys_per_graph = 4
clen = len(df_screen.columns.values)
glen = int(clen/ys_per_graph) + (1 if clen % ys_per_graph > 0 else 0)
all_y_cols = [c for c in df_screen.columns.values if c!='date']
for i in range(glen):
    b = i*ys_per_graph
    e = b + ys_per_graph
    cols = ['date'] + all_y_cols[b:e]
    iplot(dashapp.plotly_plot(df_screen[cols],x_column='date'))

In [736]:
glen

0

In [710]:
set(get_padd(df_pet_lower.columns.values))

{'padd 1',
 'padd 1a',
 'padd 1b',
 'padd 1c',
 'padd 2',
 'padd 2r',
 'padd 3',
 'padd 4',
 'padd 4 and padd 5',
 'padd 4 and west coast padd 5',
 'padd 5'}

In [478]:
set(get_padd_region(df_pet_lower.columns.values))

{'',
 'central atlantic',
 'east coast',
 'gulf coast',
 'lower atlantic',
 'midwest',
 'new england',
 'rocky mountain',
 'rocky mountains',
 'weekly gulf coast',
 'weekly rocky mountain',
 'west coast'}

In [490]:
ss =set(get_padd_source(df_pet_lower.columns.values,exclude_words=['price']))
len(ss),ss

(54,
 {'commercial crude oil imports excluding spr weekly',
  'commercial kerosene-type jet fuel weekly',
  'conventional cbob gasoline blending components weekly',
  'conventional gtab gasoline blending components weekly',
  'conventional motor gasoline ed55 and lower weekly',
  'conventional motor gasoline greater than ed55 weekly',
  'conventional motor gasoline weekly',
  'conventional motor gasoline with fuel ethanol weekly',
  'conventional other gasoline blending components imports weekly',
  'conventional other gasoline blending components weekly',
  'crude oil and petroleum products weekly',
  'crude oil weekly',
  'distillate fuel oil 0 to 15 ppm sulfur weekly',
  'distillate fuel oil greater than 15 to 500 ppm sulfur weekly',
  'distillate fuel oil greater than 2000 ppm sulfur weekly',
  'distillate fuel oil greater than 500 ppm sulfur weekly',
  'distillate fuel oil greater than 500 to 2000 ppm sulfur weekly',
  'distillate fuel oil weekly',
  'finished conventional motor g

In [494]:
ss = set(get_padd_category(df_pet_lower.columns.values,exclude_words=['price']))
# ss = [s for s in ss if len(re.findall(unit_type,s))>0]#[s for s in ss if 'blender' in s]
len(ss),ss

(24,
 {'blender net production',
  'commercial crude oil imports excluding spr weekly',
  'conventional other gasoline blending components imports weekly',
  'efiner and blender net production',
  'ending stocks',
  'ending stocks excluding spr',
  'ending stocks excluding spr and including lease stock',
  'gross inputs into refineries weekly',
  'imports',
  'imports from',
  'imports from  all countries',
  'operable crude oil distillation capacity weekly',
  'oxygenate plant production',
  'percent utilization',
  'propane and propylene ending stocks excluding propylene at terminal weekly',
  'rbob with alcohol gasoline blending components imports weekly',
  'rbob with ether gasoline blending components ending stocks weekly',
  'rbob with ether gasoline blending components imports weekly',
  'refiner and blender net input',
  'refiner and blender net production',
  'refiner blender and gas plant net production',
  'refiner net input',
  'refiner net production',
  'reformulated gaso

In [501]:
ss = set(get_padd_gasoline_type(df_pet_lower.columns.values,exclude_words=['price']))
len(ss),ss

(42,
 {'',
  'commercial crude oil imports excluding spr weekly',
  'commercial kerosene-type jet fuel weekly',
  'conventional cbob',
  'conventional gtab',
  'conventional motor',
  'conventional other',
  'crude oil and petroleum products weekly',
  'crude oil weekly',
  'distillate fuel oil 0 to 15 ppm sulfur weekly',
  'distillate fuel oil greater than 15 to 500 ppm sulfur weekly',
  'distillate fuel oil greater than 2000 ppm sulfur weekly',
  'distillate fuel oil greater than 500 ppm sulfur weekly',
  'distillate fuel oil greater than 500 to 2000 ppm sulfur weekly',
  'distillate fuel oil weekly',
  'finished conventional motor',
  'finished motor',
  'finished reformulated motor',
  'fuel ethanol weekly',
  'gross inputs into refineries weekly',
  'kerosene-type jet fuel weekly',
  'military kerosene-type jet fuel weekly',
  'motor',
  'operable crude oil distillation capacity weekly',
  'other conventional motor',
  'other finished conventional motor',
  'other finished reformu

In [508]:
ss = set(get_padd_category(df_pet_lower.columns.values,exclude_words=['price']))
ss = [re.split(UNIT_TYPE,s)[0] for s in ss]
len(ss),set(ss)

(24,
 {'',
  'blender ',
  'commercial crude oil ',
  'conventional other gasoline blending components ',
  'efiner and blender ',
  'gross ',
  'operable crude oil distillation ',
  'oxygenate plant ',
  'percent ',
  'propane and propylene ',
  'rbob with alcohol gasoline blending components ',
  'rbob with ether gasoline blending components ',
  'refiner ',
  'refiner and blender ',
  'refiner blender and gas plant ',
  'reformulated gasoline non-oxygenated '})

In [511]:
ss = set([c for c in df_pet_lower.columns.values if len(re.findall(PROCESSOR_TYPE,c))>0])
len(ss),ss

(211,
 {'central atlantic padd 1b refiner blender and gas plant net production of propane and propylene weekly',
  'east coast padd 1  refiner and blender net production of commercial kerosene-type jet fuel weekly',
  'east coast padd 1  refiner and blender net production of distillate fuel oil greater than 500 ppm sulfur weekly',
  'east coast padd 1  refiner and blender net production of military kerosene-type jet fuel weekly',
  'east coast padd 1 blender net production of finished motor gasoline weekly',
  'east coast padd 1 gross inputs into refineries weekly',
  'east coast padd 1 oxygenate plant production of fuel ethanol weekly',
  'east coast padd 1 percent utilization of refinery operable capacity weekly',
  'east coast padd 1 refiner and blender net input of conventional cbob gasoline blending components weekly',
  'east coast padd 1 refiner and blender net input of conventional gtab gasoline blending components weekly',
  'east coast padd 1 refiner and blender net input of 